# Алгоритм автоматической застройки участка

В этом задании реализован алгоритм автоматической застройки участка. Алгоритм размещает «футпринты» зданий на заданном участке с учетом различных ограничений, таких как зоны, в которых строительство запрещено, и учёт вручную заданных зданий. Он также поддерживает генерацию нескольких вариантов расположения зданий.

# Содержание
- 1 - Пакеты и настройка
- 2 - Постановка задачи
- 3 - Входные данные
  - 3.1 - Данные участка
  - 3.2 - Ручные «футпринты» зданий
- 4 - Обзор алгоритма
  - 4.1 - Предобработка данных и преобразование CRS
    - Проблема 1
  - 4.2 - Разделение зон: разрешённые vs. запрещённые
    - Проблема 2
  - 4.3 - Генерация новых «футпринтов» зданий
    - Проблема 3
  - 4.4 - Интеграция ручных «футпринтов» зданий
    - Проблема 4
- 5 - Генерация выходных данных и визуализация
  - 5.1 - Создание GeoJSON выходного файла
  - 5.2 - Визуализация плана застройки
- 6 - Генерация нескольких вариантов
  - Проблема 5: Генерация и сравнение вариантов

---

## 1 - Пакеты и настройка <a id="1"></a>

В этом разделе мы импортируем все необходимые пакеты, такие как GeoPandas, Shapely, Matplotlib, NumPy и JSON. Эти пакеты позволяют выполнять манипуляции с пространственными данными, геометрические операции, визуализацию и работу с файлами GeoJSON.

---

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, mapping
import random
import numpy as np
import json

## 2 - Постановка задачи <a id="2"></a>

Цель заключается в автоматическом распределении «футпринтов» зданий на заданном участке с учетом различных ограничений:
- Участок предоставлен в виде GeoJSON файла.
- В этом файле определены зоны с ограничениями (например, дороги, зоны с запретом строительства), обозначенные свойством (`"restriction": "no_build"`).
- При необходимости могут быть заданы вручную «футпринты» зданий, которые учитываются при автоматическом размещении.
- Новые «футпринты» зданий должны поддерживать минимальное расстояние от уже существующих (ручных) зданий и зон с ограничениями.
- Алгоритм должен генерировать один или несколько вариантов расположения зданий и выдавать комбинированный GeoJSON и изображение (PNG) плана участка.

---

## 3 - Входные данные <a id="3"></a>

### 3.1 - Данные участка <a id="3.1"></a>
Основные данные участка и зон задаются через один GeoJSON файл. Этот файл содержит:
- Разрешённые зоны (участки без ограничения `"no_build"`)
- Запрещённые зоны (участки со свойством `"restriction": "no_build"`)

### 3.2 - Ручные «футпринты» зданий <a id="3.2"></a>
Опционально вы можете предоставить отдельный GeoJSON файл с вручную заданными «футпринтами» зданий. Эти здания будут объединены с новыми и учитываться как существующие препятствия при размещении.

---

## 4 - Обзор алгоритма <a id="4"></a>

### 4.1 - Предобработка данных и преобразование CRS <a id="4.1"></a>

#### Проблема 1: Чтение GeoJSON и преобразование CRS <a id="ex01"></a>
- Считать входной GeoJSON файл.
- Определить систему координат (CRS). Если CRS равна EPSG:4326 (географические координаты), выполнить преобразование в EPSG:3857 (метры) для точных вычислений расстояний.
 
### 4.2 - Разделение зон: разрешённые vs. запрещённые <a id="4.2"></a>

#### Проблема 2: Извлечение зон с запретом строительства <a id="ex02"></a>
- Разделить входные данные на разрешённые зоны (где строительство допускается) и запрещённые зоны (с `"restriction": "no_build"`).
- Использовать объединение геометрий (через метод `union_all()`) для слияния объектов каждой категории.
- Вычислить итоговую разрешённую область, вычитая запрещённые зоны из разрешённых.

### 4.3 - Генерация новых «футпринтов» зданий <a id="4.3"></a>

#### Проблема 3: Случайное размещение с учетом ограничений по расстоянию <a id="ex03"></a>
- Генерировать случайные кандидатные точки внутри итоговой разрешённой области.
- Конструировать квадратные «футпринты» зданий, используя кандидатную точку как нижний левый угол.
- Проверять, что каждое новое здание:
  - Полностью находится в разрешённой области.
  - Соблюдает минимальное расстояние (`min_distance`) от существующих (ручных) и новых зданий.
  - Расположено не ближе, чем `min_restriction_distance` к любой запрещённой зоне.

### 4.4 - Интеграция ручных «футпринтов» зданий <a id="4.4"></a>

#### Проблема 4: Объединение ручных и автоматических зданий <a id="ex04"></a>
- Считать GeoJSON с ручными зданиями (если предоставлен) и выполнить преобразование в EPSG:3857 при необходимости.
- Объединить эти «футпринты» с новыми, учитывая их как уже существующие объекты при размещении.
- Алгоритм использует эти здания в качестве ограничений при генерации новых.

---

In [ ]:
############################################
# Вспомогательные функции
############################################

def random_point_in_allowed_area(allowed, max_attempts=1000):
    """
    Генерирует случайную точку внутри области allowed.
    """
    minx, miny, maxx, maxy = allowed.bounds
    for _ in range(max_attempts):
        x_rand = random.uniform(minx, maxx)
        y_rand = random.uniform(miny, maxy)
        p = Point(x_rand, y_rand)
        if allowed.contains(p):
            return p
    return None

In [ ]:
def generate_building_footprints(allowed_area, restricted_area, existing_buildings, num_buildings, building_size,
                                 min_distance, min_restriction_distance=0, max_attempts=10000):
    """
    Генерирует до num_buildings "футпринтов" зданий (в виде квадратов), где:
      - каждая точка задаёт нижний левый угол здания,
      - здание имеет размер building_size (в метрах),
      - между зданиями сохраняется расстояние не менее min_distance.

    Для проверки расстояния используется метод distance() между полигонами.
    """
    building_polygons = []
    attempts = 0
    while len(building_polygons) < num_buildings and attempts < max_attempts:
        candidate_point = random_point_in_allowed_area(allowed_area)
        if candidate_point is None:
            break  # Если не удалось найти точку
        
        # Строим полигон здания: нижний левый угол – candidate_point
        candidate_polygon = Polygon([
            (candidate_point.x,                 candidate_point.y),
            (candidate_point.x + building_size, candidate_point.y),
            (candidate_point.x + building_size, candidate_point.y + building_size),
            (candidate_point.x,                 candidate_point.y + building_size)
        ])
        
        # Проверка: здание должно полностью входить в разрешённую область
        if not allowed_area.contains(candidate_polygon):
            attempts += 1
            continue
        
        # Проверка: расстояние до каждого уже размещённого здания >= min_distance
        too_close = False
        for b in building_polygons:
            if candidate_polygon.distance(b) < min_distance:
                too_close = True
                break
        
        if too_close:
            attempts += 1
            continue
        
        if restricted_area and (not restricted_area.is_empty):
            if candidate_polygon.distance(restricted_area) < min_restriction_distance:
                attempts += 1
                continue
        
        building_polygons.append(candidate_polygon)
        attempts += 1

    return building_polygons

## 5 - Генерация выходных данных и визуализация <a id="5"></a>

### 5.1 - Создание GeoJSON выходного файла <a id="5.1"></a>
- Объединить исходные данные участка с новыми объектами зданий.
- Каждый новый «футпринт» здания хранится как объект GeoJSON с соответствующими свойствами (например, `"type": "residential"`, `"variant": variant_id`).
- Сохранить объединённые данные в новый GeoJSON файл (при этом итоговая система координат должна соответствовать требованиям веб-интерфейсов, обычно EPSG:4326).

### 5.2 - Визуализация плана застройки <a id="5.2"></a>
- Создать наглядное изображение (PNG) плана участка:
  - Разрешённые зоны отображаются светло-голубым.
  - Запрещённые зоны показываются серым.
  - Ручные здания (если имеются) отмечены оранжевым.
  - Новые размещённые здания выделяются красным.
- Включить легенду с понятными подписями для каждой категории.

---

## 6 - Генерация нескольких вариантов <a id="6"></a>

#### Проблема 5: Генерация и сравнение вариантов <a id="ex05"></a>
- Алгоритм поддерживает генерацию нескольких вариантов размещения зданий.
- Для каждого варианта случайное зерно (seed) изменяется, чтобы получить различные раскладки.
- Каждый вариант сохраняется как отдельный GeoJSON файл и соответствующее PNG изображение, что позволяет легко сравнить разные решения.

In [ ]:
############################################
# Основная функция обработки входного GeoJSON
############################################

def process_geojson_combined(plot_data_file,output_geojson_file,output_image_file,num_buildings, 
                             min_distance, building_size=3,min_restriction_distance=0,variant_id=0,manual_buildings_file=None):
    """
    Обрабатывает входной GeoJSON-файл (plot_data_file), содержащий как разрешённые зоны 
    (например, "Зона застройки"), так и зоны с ограничениями ("restriction": "no_build").

    Шаги:
    1. Чтение исходного файла в GeoDataFrame и определение CRS.
    2. Если входные данные в EPSG:4326, переводим их в EPSG:3857 (метры) для расчётов расстояний.
    3. Делим объекты на разрешённые зоны и зоны с ограничениями.
    4. Вычитаем ограниченные зоны из разрешённых, получаем final-область для застройки.
    5. Генерируем здания (квадраты) в EPSG:3857.
    6. Переводим здания обратно в исходную CRS (если она была EPSG:4326).
    7. Объединяем исходные объекты + новые здания в один GeoJSON.
    8. Создаём изображение (PNG) с легендой.

    Параметры:
    - plot_data_file: входной GeoJSON.
    - output_geojson_file: путь к итоговому GeoJSON.
    - output_image_file: путь к итоговому PNG.
    - num_buildings: число зданий.
    - min_distance: минимальное расстояние между зданиями (в метрах).
    - building_size: размер стороны здания (в метрах).
    - variant_id: метка варианта застройки (для записи в properties).
    """
    # Читаем исходный GeoJSON как словарь (для объединения в конце)
    with open(plot_data_file, "r", encoding="utf-8") as f:
        original_data = json.load(f)
    
    # Читаем данные через GeoPandas
    gdf = gpd.read_file(plot_data_file)
    original_crs = gdf.crs  # Сохраняем исходную систему координат (может быть None)
    
    # Переводим в EPSG:3857 (метры), если данные в EPSG:4326
    to_meter_crs = False
    if original_crs and original_crs.to_string() == "EPSG:4326":
        gdf = gdf.to_crs(epsg=3857)
        to_meter_crs = True
    
    manual_buildings_gdf_3857 = None
    if manual_buildings_file:
        manual_gdf = gpd.read_file(manual_buildings_file)
        if manual_gdf.crs and manual_gdf.crs.to_string() == "EPSG:4326":
            manual_gdf = manual_gdf.to_crs(epsg=3857)
        manual_buildings_gdf_3857 = manual_gdf
    else:
        manual_buildings_gdf_3857 = gpd.GeoDataFrame(columns=["geometry"], crs="EPSG:3857")
    
    # Разделяем зоны по наличию "restriction" == "no_build"
    if "restriction" in gdf.columns:
        restricted_gdf = gdf[gdf["restriction"] == "no_build"]
        allowed_gdf = gdf[gdf["restriction"] != "no_build"]
    else:
        restricted_gdf = gpd.GeoDataFrame(columns=gdf.columns, crs=gdf.crs)
        allowed_gdf = gdf.copy()
    
    # Объединяем геометрии разрешённых зон (union_all вместо unary_union)
    if not allowed_gdf.empty:
        allowed_area = allowed_gdf.geometry.union_all()
    else:
        allowed_area = None
    
    # Если есть зоны с ограничениями, вычитаем их из разрешённой области
    if not restricted_gdf.empty:
        restricted_area = restricted_gdf.geometry.union_all()
    else:
        restricted_area = None
        
    if allowed_area and not allowed_area.is_empty:
        if restricted_area and not restricted_area.is_empty:
            allowed_area_final = allowed_area.difference(restricted_area)
        else:
            allowed_area_final = allowed_area
    else:
        allowed_area_final = None
        
    existing_buildings_3857 = []
    if manual_buildings_gdf_3857 is not None and not manual_buildings_gdf_3857.empty:
        existing_buildings_3857 = list(manual_buildings_gdf_3857.geometry)
    
    # Генерация зданий (в метрической проекции, если to_meter_crs == True)
    building_polygons_3857 = []
    if allowed_area_final and not allowed_area_final.is_empty:
        building_polygons_3857 = generate_building_footprints(
            allowed_area=allowed_area_final,
            restricted_area=restricted_area,
            existing_buildings=existing_buildings_3857,
            num_buildings=num_buildings,
            building_size=building_size,
            min_distance=min_distance,
            min_restriction_distance=min_restriction_distance
        )
    
    # Создаём GeoDataFrame для зданий в EPSG:3857
    new_buildings_gdf_3857 = gpd.GeoDataFrame(
        {"variant": [variant_id]*len(building_polygons_3857)},
        geometry=building_polygons_3857,
        crs="EPSG:3857"
    )
    
    # Если исходные данные были в EPSG:4326, переводим здания обратно
    if to_meter_crs:
        new_buildings_gdf = new_buildings_gdf_3857.to_crs("EPSG:4326")
    else:
        # Если исходные данные не были в EPSG:4326, оставляем как есть
        new_buildings_gdf = new_buildings_gdf_3857.copy()
    
    # Формируем список Feature для новых зданий
    building_features = []
    for _, row in new_buildings_gdf.iterrows():
        feature = {
            "type": "Feature",
            "properties": {
                "type": "residential",
                "variant": row["variant"]
            },
            "geometry": mapping(row.geometry)
        }
        building_features.append(feature)
    
    # Объединяем исходные данные + новые здания в один GeoJSON
    combined_features = original_data["features"] + building_features
    combined_data = {
        "type": "FeatureCollection",
        "features": combined_features
    }
    
    # Сохраняем объединённый GeoJSON
    with open(output_geojson_file, "w", encoding="utf-8") as f:
        json.dump(combined_data, f, ensure_ascii=False, indent=2)
    print(f"Объединённый GeoJSON сохранён в файл: {output_geojson_file}")
    
    # Визуализация плана застройки (будем рисовать в проекции 3857, если есть)
    fig, ax = plt.subplots(figsize=(8,8))
    
    # Если есть разрешённая область (allowed_area)
    if allowed_area and not allowed_area.is_empty:
        if allowed_area.geom_type == "Polygon":
            x, y = allowed_area.exterior.xy
            ax.fill(x, y, alpha=0.2, fc='lightblue', ec='blue', label='Зона застройки')
        elif allowed_area.geom_type == "MultiPolygon":
            for poly in allowed_area.geoms:
                x, y = poly.exterior.xy
                ax.fill(x, y, alpha=0.2, fc='lightblue', ec='blue')
    
    # Отрисовка зон с ограничениями
    if restricted_area and not restricted_area.is_empty:
        if restricted_area.geom_type == "Polygon":
            x, y = restricted_area.exterior.xy
            ax.fill(x, y, alpha=0.5, fc='grey', ec='black', label='Запрещённая зона')
        elif restricted_area.geom_type == "MultiPolygon":
            for poly in restricted_area.geoms:
                x, y = poly.exterior.xy
                ax.fill(x, y, alpha=0.5, fc='grey', ec='black')
    
    if existing_buildings_3857:
        manual_buildings_gdf_3857.plot(ax=ax, color="orange", edgecolor="black", label="Подготовленные здания")
        
    new_buildings_gdf_3857.plot(ax=ax, color="red", edgecolor="black", label="Новые здания")
    
    ax.set_title(f"План застройки (Вариант {variant_id})")
    ax.set_xlabel("X (метры)")
    ax.set_ylabel("Y (метры)")
    
    # Создаём proxy-объекты для легенды
    import matplotlib.patches as mpatches
    import matplotlib.lines as mlines
    patch_allowed = mpatches.Patch(color='lightblue', label='Зона застройки')
    patch_restricted = mpatches.Patch(color='grey', label='Запрещённая зона')
    patch_manual = mpatches.Patch(color='orange', label='Ручные здания')
    patch_new = mpatches.Patch(color='red', label='Новые здания')
    ax.legend(handles=[patch_allowed, patch_restricted, patch_manual, patch_new])
    
    plt.savefig(output_image_file)
    plt.close()
    print(f"Изображение плана застройки сохранено в файл: {output_image_file}")

In [ ]:
############################################
# Функция для генерации нескольких вариантов
############################################

def process_geojson_multiple_variants(plot_data_file, num_variants,num_buildings, min_distance, building_size=3, 
                                      min_restriction_distance=0, manual_buildings_file=None):
    """
    Генерирует несколько вариантов расположения зданий, вызывая process_geojson_combined 
    в цикле. Каждый вариант сохраняется в отдельные GeoJSON и PNG-файлы.
    
    Параметры:
    - plot_data_file: входной GeoJSON (разрешённые зоны + ограниченные зоны).
    - num_variants: сколько вариантов хотим сгенерировать.
    - num_buildings: число зданий в каждом варианте.
    - min_distance: минимальное расстояние между зданиями (метры).
    - building_size: длина стороны здания (метры).
    """
    for variant_id in range(num_variants):
        # Меняем seed, чтобы варианты были разными
        seed_val = 42 + variant_id
        random.seed(seed_val)
        np.random.seed(seed_val)
        
        # Имена файлов для каждого варианта
        out_geojson = f"development_plan_variant_{variant_id}.geojson"
        out_image = f"development_plan_variant_{variant_id}.png"
        
        print(f"\n=== Генерация варианта {variant_id} ===")
        process_geojson_combined(
            plot_data_file=plot_data_file,
            output_geojson_file=out_geojson,
            output_image_file=out_image,
            num_buildings=num_buildings,
            min_distance=min_distance,
            building_size=building_size,
            min_restriction_distance=min_restriction_distance,
            variant_id=variant_id,
            manual_buildings_file=manual_buildings_file
        )

In [ ]:
############################################
# Пример вызова функции для нескольких вариантов
############################################

# Путь к входному GeoJSON, в котором:
#  - Есть зона застройки (без "restriction": "no_build")
#  - Есть объекты с ограничением ("restriction": "no_build")
plot_data_file = "test.geojson"

# Генерируем 3 варианта (можно указать любое число)
num_variants = 3

# Параметры застройки
num_buildings = 10     # количество зданий в каждом варианте
min_distance = 20      # минимальное расстояние между зданиями (в метрах)
building_size = 100      # размер здания (длина стороны квадрата, метры)
min_restriction_distance = 10   # минимальное расстояние до запрещённой зоны (в метрах)

# Запускаем генерацию нескольких вариантов
process_geojson_multiple_variants(
    plot_data_file=plot_data_file,
    num_variants=num_variants,
    num_buildings=num_buildings,
    min_distance=min_distance,
    building_size=building_size,
    min_restriction_distance=min_restriction_distance,
    #manual_buildings_file=manual_buildings_file   # Если есть файл с уже существующими зданиями
)